In [ ]:
# === CELL 1: SETUP ===
!pip install -q yfinance scipy

import numpy as np
import pandas as pd
import yfinance as yf
import json
import time
from datetime import datetime
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

print("✅ Setup complete")
print(f"📅 Date: {datetime.now().strftime('%Y-%m-%d %H:%M')}")

In [ ]:
# === CELL 2: CONFIGURATION ===

# Test configuration
TEST_CONFIG = {
    'train_pct': 0.70,          # 70% train, 30% test
    'episodes_per_strategy': 100, # Statistical significance
    'initial_balance': 100000,
    'max_positions': 10,
    'position_size': 0.15,       # 15% per position
    'stop_loss': -12,            # -12% stop
    'profit_target': 20,         # +20% target
}

# === YOUR HUMAN PATTERNS (Trinity + Your discoveries) ===
HUMAN_PATTERNS = {
    'dip_buy': {
        'rsi_max': 35,
        'mom_5d_max': -5,
        'description': 'RSI < 35 AND 5d momentum < -5%'
    },
    'bounce_recovery': {
        'bounce_min': 5,
        'ema8_rising': True,
        'description': 'Bounce > 5% from 5d low AND EMA8 rising'
    },
    'ribbon_momentum': {
        'ribbon_bullish': True,
        'macd_rising': True,
        'rsi_range': (50, 70),
        'description': 'Bullish ribbon + MACD rising + RSI 50-70'
    },
    'squeeze_breakout': {
        'bb_squeeze': True,
        'ribbon_tight': True,
        'ribbon_bullish': True,
        'description': 'BB squeeze + tight ribbon + bullish stack'
    }
}

# === AI DISCOVERED PATTERNS (from training results) ===
AI_PATTERNS = {
    'quantum_momentum': {
        'mom_5d_min': 10,
        'macd_rising': True,
        'bounce_signal': True,
        'description': 'Strong 5d momentum + MACD + bounce (quantum stocks)'
    },
    'nuclear_dip': {
        'ret_21d_max': -5,
        'macd_rising': True,
        'description': '21d return < -5% + MACD rising (nuclear plays)'
    },
    'volume_breakout': {
        'vol_spike': True,
        'mom_5d_min': 5,
        'rsi_max': 65,
        'description': 'Volume spike + positive momentum + RSI not overbought'
    },
    'trend_continuation': {
        'trend_align_min': 0.67,
        'ribbon_bullish': True,
        'rsi_range': (45, 70),
        'description': 'All timeframes aligned + bullish ribbon'
    }
}

WATCHLIST = [
    'APLD', 'SERV', 'MRVL', 'NVDA', 'AMD', 'MU', 'QCOM', 'CRDO',
    'SMR', 'OKLO', 'LEU', 'UUUU', 'CCJ',
    'HOOD', 'LUNR', 'SNOW', 'NOW',
    'IONQ', 'RGTI', 'QUBT',
    'TSLA', 'META', 'GOOGL',
    'SPY', 'QQQ',
    'BA', 'RIVN', 'LYFT'
]

print("🥊 PATTERN BATTLE CONFIGURATION")
print(f"   Train/Test Split: {TEST_CONFIG['train_pct']*100:.0f}% / {(1-TEST_CONFIG['train_pct'])*100:.0f}%")
print(f"   Episodes per strategy: {TEST_CONFIG['episodes_per_strategy']}")
print(f"   Human patterns: {len(HUMAN_PATTERNS)}")
print(f"   AI patterns: {len(AI_PATTERNS)}")
print(f"   Tickers: {len(WATCHLIST)}")

In [ ]:
# === CELL 3: LOAD AND SPLIT DATA ===
print("📥 Loading data...")

data_dict = {}
for ticker in WATCHLIST:
    try:
        df = yf.download(ticker, period='2y', progress=False)
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = df.columns.get_level_values(0)
        df = df.reset_index()
        for col in ['Open', 'High', 'Low', 'Close', 'Volume']:
            df[col] = pd.to_numeric(df[col], errors='coerce')
        if len(df) > 100:
            data_dict[ticker] = df
            print(f"   ✓ {ticker}: {len(df)} days")
    except Exception as e:
        print(f"   ✗ {ticker}: {e}")

# Split into train/test
train_data = {}
test_data = {}

for ticker, df in data_dict.items():
    split_idx = int(len(df) * TEST_CONFIG['train_pct'])
    train_data[ticker] = df.iloc[:split_idx].copy().reset_index(drop=True)
    test_data[ticker] = df.iloc[split_idx:].copy().reset_index(drop=True)

sample_ticker = list(data_dict.keys())[0]
print(f"\n✅ Data split complete")
print(f"   Train period: {train_data[sample_ticker]['Date'].iloc[0].strftime('%Y-%m-%d')} to {train_data[sample_ticker]['Date'].iloc[-1].strftime('%Y-%m-%d')}")
print(f"   Test period: {test_data[sample_ticker]['Date'].iloc[0].strftime('%Y-%m-%d')} to {test_data[sample_ticker]['Date'].iloc[-1].strftime('%Y-%m-%d')}")

In [ ]:
# === CELL 4: FEATURE ENGINE ===
def compute_features(df):
    """Compute all features"""
    df = df.copy()
    close = df['Close'].astype(float)
    high = df['High'].astype(float)
    low = df['Low'].astype(float)
    volume = df['Volume'].astype(float)
    
    # Returns
    for p in [1, 5, 10, 21]:
        df[f'ret_{p}d'] = close.pct_change(p) * 100
    
    # EMAs
    for p in [8, 13, 21, 34, 55]:
        df[f'ema_{p}'] = close.ewm(span=p).mean()
    
    df['ema_8_rising'] = (df['ema_8'] > df['ema_8'].shift(3)).astype(float)
    
    # Ribbon
    ribbon_cols = ['ema_8', 'ema_13', 'ema_21', 'ema_34', 'ema_55']
    df['ribbon_min'] = df[ribbon_cols].min(axis=1)
    df['ribbon_max'] = df[ribbon_cols].max(axis=1)
    df['ribbon_range'] = (df['ribbon_max'] - df['ribbon_min']) / (df['ribbon_min'] + 1e-10) * 100
    df['ribbon_bullish'] = ((df['ema_8'] > df['ema_13']) & (df['ema_13'] > df['ema_21'])).astype(float)
    df['ribbon_tight'] = (df['ribbon_range'] < 5).astype(float)
    
    # RSI
    delta = close.diff()
    gain = delta.where(delta > 0, 0).rolling(14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(14).mean()
    rs = gain / (loss + 1e-10)
    df['rsi_14'] = 100 - (100 / (1 + rs))
    
    # MACD
    ema12 = close.ewm(span=12).mean()
    ema26 = close.ewm(span=26).mean()
    df['macd'] = ema12 - ema26
    df['macd_signal'] = df['macd'].ewm(span=9).mean()
    df['macd_hist'] = df['macd'] - df['macd_signal']
    df['macd_rising'] = (df['macd_hist'] > df['macd_hist'].shift(1)).astype(float)
    
    # Bollinger
    mid = close.rolling(20).mean()
    std = close.rolling(20).std()
    df['bb_width'] = (4 * std) / (mid + 1e-10) * 100
    df['bb_squeeze'] = (df['bb_width'] < df['bb_width'].rolling(50).mean() * 0.75).astype(float)
    
    # Volume
    df['vol_sma'] = volume.rolling(20).mean()
    df['vol_ratio'] = volume / (df['vol_sma'] + 1)
    df['vol_spike'] = (df['vol_ratio'] > 2.0).astype(float)
    
    # Momentum
    df['mom_5d'] = close.pct_change(5) * 100
    
    # Bounce
    df['low_5d'] = low.rolling(5).min()
    df['bounce'] = (close / (df['low_5d'] + 1e-10) - 1) * 100
    df['bounce_signal'] = ((df['bounce'] > 3) & (df['ema_8_rising'] > 0)).astype(float)
    
    # Trend alignment
    df['trend_5d'] = np.sign(df['ret_5d'])
    df['trend_10d'] = np.sign(df['ret_10d'])
    df['trend_21d'] = np.sign(df['ret_21d'])
    df['trend_align'] = (df['trend_5d'] + df['trend_10d'] + df['trend_21d']) / 3
    
    df = df.replace([np.inf, -np.inf], np.nan)
    df = df.ffill().bfill().fillna(0)
    
    return df

print("🧠 Computing features...")
train_features = {t: compute_features(df) for t, df in train_data.items()}
test_features = {t: compute_features(df) for t, df in test_data.items()}
print("✅ Features computed for train and test sets")

In [ ]:
# === CELL 5: PATTERN MATCHERS ===

def check_human_patterns(df, idx):
    """Check if any HUMAN pattern triggers a buy signal"""
    rsi = float(df['rsi_14'].iloc[idx])
    mom_5d = float(df['mom_5d'].iloc[idx])
    bounce = float(df['bounce'].iloc[idx])
    ema8_rising = float(df['ema_8_rising'].iloc[idx])
    ribbon_bullish = float(df['ribbon_bullish'].iloc[idx])
    ribbon_tight = float(df['ribbon_tight'].iloc[idx])
    macd_rising = float(df['macd_rising'].iloc[idx])
    bb_squeeze = float(df['bb_squeeze'].iloc[idx])
    
    signals = []
    
    # DIP BUY
    if rsi < 35 and mom_5d < -5:
        signals.append('H:dip_buy')
    
    # BOUNCE RECOVERY
    if bounce > 5 and ema8_rising > 0:
        signals.append('H:bounce')
    
    # RIBBON MOMENTUM
    if ribbon_bullish > 0 and macd_rising > 0 and 50 < rsi < 70:
        signals.append('H:ribbon_mom')
    
    # SQUEEZE BREAKOUT
    if bb_squeeze > 0 and ribbon_tight > 0 and ribbon_bullish > 0:
        signals.append('H:squeeze')
    
    return signals


def check_ai_patterns(df, idx):
    """Check if any AI-discovered pattern triggers a buy signal"""
    rsi = float(df['rsi_14'].iloc[idx])
    mom_5d = float(df['mom_5d'].iloc[idx])
    ret_21d = float(df['ret_21d'].iloc[idx])
    macd_rising = float(df['macd_rising'].iloc[idx])
    bounce_signal = float(df['bounce_signal'].iloc[idx])
    vol_spike = float(df['vol_spike'].iloc[idx])
    trend_align = float(df['trend_align'].iloc[idx])
    ribbon_bullish = float(df['ribbon_bullish'].iloc[idx])
    
    signals = []
    
    # QUANTUM MOMENTUM (AI discovered)
    if mom_5d > 10 and macd_rising > 0 and bounce_signal > 0:
        signals.append('AI:quantum_mom')
    
    # NUCLEAR DIP (AI discovered)
    if ret_21d < -5 and macd_rising > 0:
        signals.append('AI:nuclear_dip')
    
    # VOLUME BREAKOUT (AI discovered)
    if vol_spike > 0 and mom_5d > 5 and rsi < 65:
        signals.append('AI:vol_break')
    
    # TREND CONTINUATION (AI discovered)
    if trend_align >= 0.67 and ribbon_bullish > 0 and 45 < rsi < 70:
        signals.append('AI:trend_cont')
    
    return signals


def check_combined_patterns(df, idx):
    """Check BOTH human and AI patterns"""
    return check_human_patterns(df, idx) + check_ai_patterns(df, idx)


print("✅ Pattern matchers ready")
print(f"   Human patterns: {list(HUMAN_PATTERNS.keys())}")
print(f"   AI patterns: {list(AI_PATTERNS.keys())}")

In [ ]:
# === CELL 6: TRADING SIMULATOR ===

def run_backtest(features_dict, data_dict, pattern_checker, exploration=0.0):
    """Run a single backtest with given pattern checker"""
    balance = TEST_CONFIG['initial_balance']
    positions = {}
    trades = []
    history = [balance]
    pattern_stats = {}  # Track which patterns work
    
    tickers = list(features_dict.keys())
    min_len = min(len(df) for df in data_dict.values())
    
    for day in range(60, min_len - 1):
        prices = {}
        
        # Update positions
        for t in list(positions.keys()):
            if t in data_dict and day < len(data_dict[t]):
                price = float(data_dict[t]['Close'].iloc[day])
                prices[t] = price
                positions[t]['days'] += 1
                if price > positions[t]['max']:
                    positions[t]['max'] = price
        
        # Process each ticker
        for ticker in tickers:
            if day >= len(features_dict[ticker]):
                continue
            
            df = features_dict[ticker]
            price = float(data_dict[ticker]['Close'].iloc[day])
            prices[ticker] = price
            
            # SELL LOGIC
            if ticker in positions:
                pos = positions[ticker]
                pnl = (price / pos['entry'] - 1) * 100
                from_max = (price / pos['max'] - 1) * 100
                
                sell = False
                if pnl >= TEST_CONFIG['profit_target']:
                    sell = True
                elif pnl <= TEST_CONFIG['stop_loss']:
                    sell = True
                elif pnl > 15 and from_max < -8:
                    sell = True
                elif pos['days'] > 60 and pnl > 0:
                    sell = True
                
                if sell:
                    balance += pos['shares'] * price
                    trade = {
                        'ticker': ticker,
                        'pnl': pnl / 100,
                        'days': pos['days'],
                        'patterns': pos['patterns']
                    }
                    trades.append(trade)
                    
                    # Track pattern performance
                    for p in pos['patterns']:
                        if p not in pattern_stats:
                            pattern_stats[p] = {'wins': 0, 'losses': 0, 'total_pnl': 0}
                        if pnl > 0:
                            pattern_stats[p]['wins'] += 1
                        else:
                            pattern_stats[p]['losses'] += 1
                        pattern_stats[p]['total_pnl'] += pnl
                    
                    del positions[ticker]
            
            # BUY LOGIC
            else:
                if len(positions) >= TEST_CONFIG['max_positions']:
                    continue
                
                pv = balance + sum(positions[t]['shares'] * prices.get(t, 0) for t in positions)
                if balance / pv < 0.1:
                    continue
                
                # Check patterns
                signals = pattern_checker(df, day)
                
                # Add exploration
                buy = len(signals) > 0
                if exploration > 0 and np.random.random() < exploration:
                    buy = True
                    signals = ['EXPLORE']
                
                if buy:
                    shares = int(balance * TEST_CONFIG['position_size'] / price)
                    if shares > 0:
                        balance -= shares * price
                        positions[ticker] = {
                            'shares': shares,
                            'entry': price,
                            'max': price,
                            'days': 0,
                            'patterns': signals
                        }
        
        # Track portfolio value
        pv = balance + sum(positions[t]['shares'] * prices.get(t, 0) for t in positions)
        history.append(pv)
    
    # Liquidate remaining
    for ticker, pos in positions.items():
        if ticker in data_dict:
            price = float(data_dict[ticker]['Close'].iloc[-1])
            balance += pos['shares'] * price
            pnl = (price / pos['entry'] - 1)
            trades.append({
                'ticker': ticker,
                'pnl': pnl,
                'days': pos['days'],
                'patterns': pos['patterns']
            })
    
    # Calculate metrics
    total_return = (balance / TEST_CONFIG['initial_balance'] - 1) * 100
    winners = [t for t in trades if t['pnl'] > 0]
    losers = [t for t in trades if t['pnl'] <= 0]
    win_rate = len(winners) / max(len(trades), 1) * 100
    
    # Sharpe ratio
    returns = np.diff(history) / history[:-1]
    sharpe = np.mean(returns) / (np.std(returns) + 1e-10) * np.sqrt(252) if len(returns) > 0 else 0
    
    # Max drawdown
    peak = np.maximum.accumulate(history)
    drawdown = (np.array(history) - peak) / (peak + 1e-10) * 100
    max_dd = np.min(drawdown)
    
    return {
        'total_return': total_return,
        'trades': len(trades),
        'win_rate': win_rate,
        'avg_winner': np.mean([t['pnl'] for t in winners]) * 100 if winners else 0,
        'avg_loser': np.mean([t['pnl'] for t in losers]) * 100 if losers else 0,
        'sharpe': sharpe,
        'max_drawdown': max_dd,
        'pattern_stats': pattern_stats
    }

print("✅ Trading simulator ready")

In [ ]:
# === CELL 7: RUN PATTERN BATTLE - TRAINING SET ===
print("=" * 70)
print("🏋️ PHASE 1: TRAINING SET RESULTS")
print("=" * 70)

strategies = {
    '👤 HUMAN': check_human_patterns,
    '🤖 AI': check_ai_patterns,
    '🔥 COMBINED': check_combined_patterns
}

train_results = {}

for name, checker in strategies.items():
    print(f"\n{name} Strategy:")
    results = []
    
    for ep in range(TEST_CONFIG['episodes_per_strategy']):
        # Add some exploration to discover patterns
        exploration = 0.1 if ep < 50 else 0.0
        result = run_backtest(train_features, train_data, checker, exploration)
        results.append(result)
        
        if ep % 25 == 0:
            print(f"   Episode {ep}: Return {result['total_return']:+.1f}%, WR {result['win_rate']:.0f}%")
    
    # Aggregate results
    train_results[name] = {
        'avg_return': np.mean([r['total_return'] for r in results]),
        'std_return': np.std([r['total_return'] for r in results]),
        'avg_win_rate': np.mean([r['win_rate'] for r in results]),
        'avg_sharpe': np.mean([r['sharpe'] for r in results]),
        'avg_max_dd': np.mean([r['max_drawdown'] for r in results]),
        'best_return': max([r['total_return'] for r in results]),
        'worst_return': min([r['total_return'] for r in results]),
        'all_results': results
    }

print("\n" + "=" * 70)
print("📊 TRAINING SET SUMMARY")
print("=" * 70)
print(f"{'Strategy':<15} {'Avg Return':>12} {'Std Dev':>10} {'Win Rate':>10} {'Sharpe':>8} {'Max DD':>10}")
print("-" * 70)
for name, r in train_results.items():
    print(f"{name:<15} {r['avg_return']:>+11.1f}% {r['std_return']:>9.1f}% {r['avg_win_rate']:>9.0f}% {r['avg_sharpe']:>7.2f} {r['avg_max_dd']:>9.1f}%")

In [ ]:
# === CELL 8: RUN PATTERN BATTLE - TEST SET (UNSEEN DATA!) ===
print("=" * 70)
print("🧪 PHASE 2: TEST SET RESULTS (UNSEEN DATA!)")
print("=" * 70)
print("⚠️  This is the REAL test - patterns must work on data never seen!")

test_results = {}

for name, checker in strategies.items():
    print(f"\n{name} Strategy:")
    results = []
    
    for ep in range(TEST_CONFIG['episodes_per_strategy']):
        # NO exploration on test set - pure pattern performance
        result = run_backtest(test_features, test_data, checker, exploration=0.0)
        results.append(result)
        
        if ep % 25 == 0:
            print(f"   Episode {ep}: Return {result['total_return']:+.1f}%, WR {result['win_rate']:.0f}%")
    
    # Aggregate results
    test_results[name] = {
        'avg_return': np.mean([r['total_return'] for r in results]),
        'std_return': np.std([r['total_return'] for r in results]),
        'avg_win_rate': np.mean([r['win_rate'] for r in results]),
        'avg_sharpe': np.mean([r['sharpe'] for r in results]),
        'avg_max_dd': np.mean([r['max_drawdown'] for r in results]),
        'best_return': max([r['total_return'] for r in results]),
        'worst_return': min([r['total_return'] for r in results]),
        'all_results': results
    }

print("\n" + "=" * 70)
print("📊 TEST SET SUMMARY (TRUE PERFORMANCE!)")
print("=" * 70)
print(f"{'Strategy':<15} {'Avg Return':>12} {'Std Dev':>10} {'Win Rate':>10} {'Sharpe':>8} {'Max DD':>10}")
print("-" * 70)
for name, r in test_results.items():
    print(f"{name:<15} {r['avg_return']:>+11.1f}% {r['std_return']:>9.1f}% {r['avg_win_rate']:>9.0f}% {r['avg_sharpe']:>7.2f} {r['avg_max_dd']:>9.1f}%")

In [ ]:
# === CELL 9: STATISTICAL SIGNIFICANCE TEST ===
print("=" * 70)
print("📈 STATISTICAL ANALYSIS: Is the difference REAL or LUCK?")
print("=" * 70)

# Extract return arrays
human_returns = [r['total_return'] for r in test_results['👤 HUMAN']['all_results']]
ai_returns = [r['total_return'] for r in test_results['🤖 AI']['all_results']]
combined_returns = [r['total_return'] for r in test_results['🔥 COMBINED']['all_results']]

# T-tests
print("\n🔬 T-Test Results (p < 0.05 = statistically significant):")
print("-" * 50)

# Human vs AI
t_stat, p_value = stats.ttest_ind(human_returns, ai_returns)
winner = '👤 HUMAN' if np.mean(human_returns) > np.mean(ai_returns) else '🤖 AI'
sig = '✅ SIGNIFICANT' if p_value < 0.05 else '❌ NOT SIGNIFICANT (could be luck)'
print(f"\nHuman vs AI:")
print(f"   Winner: {winner}")
print(f"   p-value: {p_value:.4f}")
print(f"   Result: {sig}")

# Human vs Combined
t_stat, p_value = stats.ttest_ind(human_returns, combined_returns)
winner = '👤 HUMAN' if np.mean(human_returns) > np.mean(combined_returns) else '🔥 COMBINED'
sig = '✅ SIGNIFICANT' if p_value < 0.05 else '❌ NOT SIGNIFICANT (could be luck)'
print(f"\nHuman vs Combined:")
print(f"   Winner: {winner}")
print(f"   p-value: {p_value:.4f}")
print(f"   Result: {sig}")

# AI vs Combined
t_stat, p_value = stats.ttest_ind(ai_returns, combined_returns)
winner = '🤖 AI' if np.mean(ai_returns) > np.mean(combined_returns) else '🔥 COMBINED'
sig = '✅ SIGNIFICANT' if p_value < 0.05 else '❌ NOT SIGNIFICANT (could be luck)'
print(f"\nAI vs Combined:")
print(f"   Winner: {winner}")
print(f"   p-value: {p_value:.4f}")
print(f"   Result: {sig}")

# Overall winner
print("\n" + "=" * 70)
best_strategy = max(test_results.keys(), key=lambda x: test_results[x]['avg_return'])
print(f"🏆 OVERALL WINNER ON UNSEEN DATA: {best_strategy}")
print(f"   Average Return: {test_results[best_strategy]['avg_return']:+.1f}%")
print(f"   Consistency (Std Dev): {test_results[best_strategy]['std_return']:.1f}%")
print("=" * 70)

In [ ]:
# === CELL 10: INDIVIDUAL PATTERN ANALYSIS ===
print("=" * 70)
print("🔍 INDIVIDUAL PATTERN PERFORMANCE")
print("=" * 70)

# Aggregate pattern stats from test results
all_pattern_stats = {}

for name in strategies.keys():
    for result in test_results[name]['all_results']:
        for pattern, stats_dict in result['pattern_stats'].items():
            if pattern not in all_pattern_stats:
                all_pattern_stats[pattern] = {'wins': 0, 'losses': 0, 'total_pnl': 0}
            all_pattern_stats[pattern]['wins'] += stats_dict['wins']
            all_pattern_stats[pattern]['losses'] += stats_dict['losses']
            all_pattern_stats[pattern]['total_pnl'] += stats_dict['total_pnl']

# Print pattern performance
print(f"\n{'Pattern':<20} {'Wins':>6} {'Losses':>8} {'Win Rate':>10} {'Total PnL':>12}")
print("-" * 60)

sorted_patterns = sorted(all_pattern_stats.items(), 
                         key=lambda x: x[1]['total_pnl'], reverse=True)

for pattern, stats_dict in sorted_patterns:
    total = stats_dict['wins'] + stats_dict['losses']
    if total > 0:
        wr = stats_dict['wins'] / total * 100
        emoji = '🟢' if wr >= 55 and stats_dict['total_pnl'] > 0 else ('🔴' if wr < 45 else '🟡')
        print(f"{emoji} {pattern:<18} {stats_dict['wins']:>6} {stats_dict['losses']:>8} {wr:>9.0f}% {stats_dict['total_pnl']:>+11.1f}%")

# Best patterns
print("\n" + "=" * 70)
print("🌟 TOP 5 MOST PROFITABLE PATTERNS:")
for i, (pattern, stats_dict) in enumerate(sorted_patterns[:5], 1):
    total = stats_dict['wins'] + stats_dict['losses']
    wr = stats_dict['wins'] / total * 100 if total > 0 else 0
    print(f"   {i}. {pattern}: +{stats_dict['total_pnl']:.1f}% ({wr:.0f}% win rate)")

print("\n💀 WORST PATTERNS (avoid these!):")
for i, (pattern, stats_dict) in enumerate(sorted_patterns[-3:], 1):
    total = stats_dict['wins'] + stats_dict['losses']
    wr = stats_dict['wins'] / total * 100 if total > 0 else 0
    if stats_dict['total_pnl'] < 0:
        print(f"   {i}. {pattern}: {stats_dict['total_pnl']:.1f}% ({wr:.0f}% win rate)")

In [ ]:
# === CELL 11: GENERATE RECOMMENDATIONS ===
print("=" * 70)
print("📋 FINAL RECOMMENDATIONS")
print("=" * 70)

# Determine best strategy
test_avg_returns = {k: v['avg_return'] for k, v in test_results.items()}
best = max(test_avg_returns, key=test_avg_returns.get)
train_avg_returns = {k: v['avg_return'] for k, v in train_results.items()}

print(f"\n🏆 RECOMMENDED STRATEGY: {best}")
print(f"")
print(f"📊 Performance Summary:")
print(f"   Training Return: {train_results[best]['avg_return']:+.1f}%")
print(f"   Test Return: {test_results[best]['avg_return']:+.1f}%")
print(f"   Win Rate: {test_results[best]['avg_win_rate']:.0f}%")
print(f"   Sharpe Ratio: {test_results[best]['avg_sharpe']:.2f}")
print(f"   Max Drawdown: {test_results[best]['avg_max_dd']:.1f}%")

# Check for overfitting
print(f"\n⚠️  OVERFITTING CHECK:")
for name in strategies.keys():
    train_ret = train_results[name]['avg_return']
    test_ret = test_results[name]['avg_return']
    diff = train_ret - test_ret
    if diff > 50:
        print(f"   {name}: ❌ OVERFIT! Train {train_ret:+.0f}% vs Test {test_ret:+.0f}% (gap: {diff:.0f}%)")
    elif diff > 20:
        print(f"   {name}: ⚠️ Possible overfit. Train {train_ret:+.0f}% vs Test {test_ret:+.0f}%")
    else:
        print(f"   {name}: ✅ Generalizes well! Train {train_ret:+.0f}% vs Test {test_ret:+.0f}%")

# Best patterns to use
print(f"\n🎯 USE THESE PATTERNS:")
good_patterns = [p for p, s in sorted_patterns if s['total_pnl'] > 10 and s['wins'] + s['losses'] > 5]
for p in good_patterns[:6]:
    print(f"   ✅ {p}")

print(f"\n❌ AVOID THESE PATTERNS:")
bad_patterns = [p for p, s in sorted_patterns if s['total_pnl'] < -10]
for p in bad_patterns[:3]:
    print(f"   ❌ {p}")

print("\n" + "=" * 70)

In [ ]:
# === CELL 12: SAVE BATTLE RESULTS ===
battle_results = {
    'generated_at': datetime.now().isoformat(),
    'config': TEST_CONFIG,
    'train_results': {
        name: {k: v for k, v in r.items() if k != 'all_results'}
        for name, r in train_results.items()
    },
    'test_results': {
        name: {k: v for k, v in r.items() if k != 'all_results'}
        for name, r in test_results.items()
    },
    'winner': best,
    'pattern_performance': {
        p: {
            'wins': s['wins'],
            'losses': s['losses'],
            'win_rate': s['wins'] / (s['wins'] + s['losses']) * 100 if s['wins'] + s['losses'] > 0 else 0,
            'total_pnl': s['total_pnl']
        }
        for p, s in all_pattern_stats.items()
    },
    'recommendations': {
        'best_strategy': best,
        'use_patterns': good_patterns[:6],
        'avoid_patterns': bad_patterns[:3]
    }
}

with open('pattern_battle_results.json', 'w') as f:
    json.dump(battle_results, f, indent=2, default=str)

print("✅ Results saved to pattern_battle_results.json")

try:
    from google.colab import files
    files.download('pattern_battle_results.json')
    print("📥 Download started!")
except:
    print("(Not in Colab - file saved locally)")